In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
train_dataset = np.genfromtxt('resources/train.csv', dtype=np.float32, delimiter=',')
train_targets = train_dataset[:,-1]
train_targets = np.array([1 if x == 2 else 0 for x in train_targets])
test_dataset = np.genfromtxt('resources/test.csv', dtype=np.float32, delimiter=',')
test_targets = test_dataset[:,-1]
test_targets = np.array([1 if x == 2 else 0 for x in test_targets])
combined_dataset=np.concatenate((train_dataset[:,:-1], test_dataset[:,:-1]))

In [3]:
np.bincount(train_targets)

array([58630, 67343])

In [4]:
numeric_indices = []
for i in range(combined_dataset.shape[1]):
    n_unique = len(np.unique(combined_dataset[:, i]))
    numeric_indices.append(n_unique > 2)
print (len(numeric_indices))
numeric_indices = np.array(numeric_indices, dtype=np.bool)

122


In [5]:
combined_dataset[:, numeric_indices] = StandardScaler().fit_transform(
                                        combined_dataset[:, numeric_indices]
                                        )

In [61]:
batch_size = 1500
num_feature = 122
num_labels = 2
graph = tf.Graph()
with graph.as_default():
    target = tf.placeholder(tf.float32, shape=(None, 2))
    n_input = num_feature
    n_hidden_1 = 200
    n_hidden_2 = 50
    n_hidden_3 = 2
    
    inputs = {'l1': tf.placeholder(tf.float32, shape=(None, num_feature)),
              'l2': tf.placeholder(tf.float32, shape=(None, n_hidden_1)),
              'l3': tf.placeholder(tf.float32, shape=(None, n_hidden_2))}
    
    targets = {'l1': tf.placeholder(tf.float32, shape=(None, num_feature)),
               'l2': tf.placeholder(tf.float32, shape=(None, n_hidden_1)),
               'l3': tf.placeholder(tf.float32, shape=(None, num_labels))}

    weights = {'l1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], dtype=tf.float32)),
               'l2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], dtype=tf.float32)),
               'l3': tf.Variable(tf.truncated_normal([n_hidden_2, num_labels], dtype=tf.float32))}

    enc_biases = {'l1': tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32)),
                  'l2': tf.Variable(tf.truncated_normal([n_hidden_2], dtype=tf.float32)),
                  'l3': tf.Variable(tf.truncated_normal([num_labels], dtype=tf.float32))}
    
    dec_biases = {'l1': tf.Variable(tf.truncated_normal([n_input], dtype=tf.float32)),
                  'l2': tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32))}
    
    def encode(layer_id):
        return tf.add(tf.matmul(inputs[layer_id], weights[layer_id]), enc_biases[layer_id])
    
    def decode(layer, layer_id):
        return tf.add(tf.matmul(layer, tf.transpose(weights[layer_id])), dec_biases[layer_id])
    
    def get_loss(inp, out):
        difference = tf.sub(inp, out) 
        return tf.sqrt(tf.reduce_mean(tf.square(difference)))   

    def encoder_loss(layer_id):
        enc = encode(layer_id)
        dec = decode(enc, layer_id)
        loss = get_loss(targets[layer_id], dec)
        return loss
    
    def compute_logit(layer_id):
        logits = tf.nn.tanh(encode(layer_id))
        logits = tf.add(tf.matmul(logits, wo), bo)
        return logits
    
    def logit_loss(logits, layer_id):
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits, targets[layer_id]))
    
    def full_compute():
        logits = inputs['l1']
        for layer_id in ['l1', 'l2', 'l3']:
            logits = tf.add(tf.matmul(logits, weights[layer_id]), enc_biases[layer_id])
        return logits
        
    def get_sigmoid(logits):
        return tf.nn.sigmoid(logits)
    
    loss_l1 = encoder_loss('l1')
    optimizer_l1 = tf.train.AdamOptimizer(0.001).minimize(loss_l1)
    loss_l2 = encoder_loss('l2')
    optimizer_l2 = tf.train.AdamOptimizer(0.001).minimize(loss_l2)
    logits = full_compute()
    loss_l3 = logit_loss(logits, 'l3')
    optimizer_l3 = tf.train.AdamOptimizer(0.001).minimize(loss_l3 + 0.001*tf.nn.l2_loss(weights['l3']))

In [ ]:
num_steps = 8500
randomized_dataset = combined_dataset.copy()
np.random.shuffle(randomized_dataset)
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(1, num_steps + 1):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)
        batch_data = randomized_dataset[offset:(offset + batch_size), :]
        noisy_batch_data = batch_data.copy()
        for i, point in enumerate(noisy_batch_data):
            noisy_batch_data[i, :] = point + np.random.normal(0, 1, noisy_batch_data.shape[1])
        feed_dict = {inputs['l1']: noisy_batch_data, targets['l1']: batch_data}
        _, l = session.run([optimizer_l1, loss_l1], feed_dict=feed_dict)
        if step % 500 == 0:
            print step, ':', l

Initialized
500 : 86.2553
1000 : 36.645
1500 : 16.2874
2000 : 6.86966
2500 : 2.96181
3000 : 1.29328
3500 : 0.580485
4000 : 0.379386
4500 : 0.318181
5000 : 0.316957
5500 : 0.29658
6000 : 0.323883
6500 : 0.307271
7000 : 0.290413
7500 : 0.324015
8000 : 0.314294
8500 : 0.28926
9000 : 0.296604
9500 : 0.298376
10000 : 0.315302


In [ ]:
num_steps = 18000
preds = []
randomized_dataset = combined_dataset.copy()
np.random.shuffle(randomized_dataset)
new_input=[]
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    encoder_op = encode('l1')
    encoded_randomized_dataset = encoder_op.eval(feed_dict={inputs['l1']: randomized_dataset})
    for step in range(1, num_steps + 1):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)
        batch_data = encoded_randomized_dataset[offset:(offset + batch_size), :]
        noisy_batch_data = batch_data.copy()
        for i, point in enumerate(noisy_batch_data):
            noisy_batch_data[i, :] = point + np.random.normal(0, 1, noisy_batch_data.shape[1])
        feed_dict = {inputs['l2']: noisy_batch_data, targets['l2']: batch_data}
        _, l = session.run([optimizer_l2, loss_l2], feed_dict=feed_dict)
        if step % 1000 == 0:
            print step, ':', l

Initialized
1000 : 79.9693
2000 : 50.1782
3000

In [57]:
num_steps = 1000
preds = []
new_input=[]
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    encoded_dataset = combined_dataset[:train_dataset.shape[0], :]
    for step in range(1, num_steps + 1):
        offset = (step * batch_size) % (encoded_dataset.shape[0] - batch_size)
        batch_data = encoded_dataset[offset:(offset + batch_size), :]
        batch_labels = list(train_targets[offset:(offset + batch_size)])
        for i, label in enumerate(batch_labels):
            batch_labels[i] = (label == np.arange(2)).astype(np.int32)
        batch_labels = np.array(batch_labels, dtype=np.int32)
        feed_dict = {inputs['l1']: batch_data, targets['l3']: batch_labels}
        _, l = session.run([optimizer_l3, loss_l3], feed_dict=feed_dict)
        if step % 100 == 0:
            print step, ':', l

Initialized
1 : 57.932
2 : 56.0853
3 : 50.5863
4 : 46.7615
5 : 41.7852
6 : 40.8184
7 : 40.1066
8 : 39.5069
9 : 34.528
10 : 35.5638
11 : 36.0201
12 : 31.2986
13 : 30.3697
14 : 22.598
15 : 27.5842
16 : 23.7671
17 : 27.3785
18 : 19.7599
19 : 16.8592
20 : 20.5611
21 : 22.1059
22 : 19.4908
23 : 19.7528
24 : 18.9394
25 : 14.1991
26 : 14.8422
27 : 13.1288
28 : 18.2879
29 : 20.5551
30 : 16.562
31 : 13.8016
32 : 16.4818
33 : 13.8816
34 : 12.9311
35 : 12.6798
36 : 11.5377
37 : 10.5968
38 : 12.1328
39 : 12.4286
40 : 13.087
41 : 10.0677
42 : 9.89639
43 : 9.85662
44 : 9.51702
45 : 9.87033
46 : 11.9953
47 : 8.34478
48 : 8.20797
49 : 7.98213
50 : 8.61769
51 : 8.65382
52 : 7.28106
53 : 5.75337
54 : 7.16362
55 : 8.79215
56 : 6.50359
57 : 6.71431
58 : 9.2741
59 : 5.79919
60 : 6.46822
61 : 7.93393
62 : 5.66905
63 : 6.59138
64 : 6.72521
65 : 5.57835
66 : 7.70327
67 : 4.98589
68 : 7.75177
69 : 5.21251
70 : 5.04132
71 : 5.16483
72 : 4.34731
73 : 4.92371
74 : 5.75147
75 : 4.69978
76 : 4.62378
77 : 5.02925
78

In [58]:
preds = []
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    logit_op = full_compute()
    softmaxes = get_sigmoid(logit_op)
    preds = softmaxes.eval(feed_dict={inputs['l1']: combined_dataset[train_dataset.shape[0]:, :]})

In [59]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == labels)
          / predictions.shape[0])

In [60]:
accuracy(preds, test_targets)

71.94375443577005

In [51]:
preds

array([[ 0.82571083,  0.30747864],
       [ 0.8436923 ,  0.26298583],
       [ 0.8436923 ,  0.26298583],
       ..., 
       [ 0.77494293,  0.26730353],
       [ 0.8436923 ,  0.26298583],
       [ 0.82551992,  0.30795208]], dtype=float32)

In [50]:
test_targets

array([0, 0, 1, ..., 0, 1, 0])

In [79]:
num_steps = 7000
preds = []
randomized_dataset = combined_dataset.copy()
#     np.random.shuffle(randomized_dataset)
new_input=[]
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)
        batch_data = randomized_dataset[offset:(offset + batch_size), :]
        noisy_batch_data = batch_data.copy()
        for i, point in enumerate(noisy_batch_data):
            noisy_batch_data[i, :] = point + np.random.normal(0, 1, noisy_batch_data.shape[1])
        feed_dict = {input_data: noisy_batch_data, clean_data: batch_data}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
#             print "SKLEARN loss", np.sqrt(mean_squared_error(predictions, batch_data))
    new_input=layer_1.eval(feed_dict = {input_data :combined_dataset})
    test_targets_1hot = np.array([(x == np.arange(2)).astype(np.float32) for x in test_targets])
#     print(test_targets_1hot[:5])
    for step in range(370):
        c = np.random.choice(train_targets.shape[0], 2000, replace=False)
        batch_data = train_dataset[:,:-1][c]
        batch_targets = train_targets[c]
        batch_targets=np.array([(x == np.arange(2)).astype(np.float32) for x in batch_targets])
        feed_dict = {clean_data:batch_data, target:batch_targets}
        l, _ = session.run([loss_, optimize_], feed_dict=feed_dict)
        if (step % 10 == 0):
            print("Minibatch logloss at step", step, ":", l)
    new_input = layer_1_f.eval(feed_dict={clean_data:combined_dataset})
    print(session.run(a_, feed_dict={clean_data:combined_dataset[len(train_targets):],target:test_targets_1hot}))

Initialized
('Minibatch loss at step', 0, ':', 222.77254)
('Minibatch loss at step', 500, ':', 89.292938)
('Minibatch loss at step', 1000, ':', 39.540432)
('Minibatch loss at step', 1500, ':', 16.709871)
('Minibatch loss at step', 2000, ':', 26.412645)
('Minibatch loss at step', 2500, ':', 3.530025)
('Minibatch loss at step', 3000, ':', 1.5838422)
('Minibatch loss at step', 3500, ':', 0.6859898)
('Minibatch loss at step', 4000, ':', 0.40181309)
('Minibatch loss at step', 4500, ':', 0.38369343)
('Minibatch loss at step', 5000, ':', 0.31740069)
('Minibatch loss at step', 5500, ':', 0.33525366)
('Minibatch loss at step', 6000, ':', 0.36136743)
('Minibatch loss at step', 6500, ':', 0.29976884)
('Minibatch logloss at step', 0, ':', 2.0832458)
('Minibatch logloss at step', 10, ':', 0.43901727)
('Minibatch logloss at step', 20, ':', 0.27650648)
('Minibatch logloss at step', 30, ':', 0.19391036)
('Minibatch logloss at step', 40, ':', 0.22965193)
('Minibatch logloss at step', 50, ':', 0.1282324

In [36]:
#saving new train & test data 
from collections import Counter
print(train_dataset.shape)
new_train_data=new_input[:train_dataset.shape[0]]
new_train_data=np.c_[new_train_data,train_targets]
new_test_data=new_input[train_dataset.shape[0]:]
new_test_data=np.c_[new_test_data, test_targets]
print(new_train_data.shape, new_test_data.shape)
# np.savez('out/new_input.npz',train=new_train_data, test=new_test_data)

(125973, 123)
((125973, 201), (22544, 201))


In [37]:
model = LogisticRegression(C=1,class_weight='balanced',n_jobs=-1)

In [37]:
print (cross_val_score(model, new_train_data[:, :-1], train_targets, cv=3, scoring='precision_macro').mean())

0.731867870076


In [45]:
new_train_data = np.load('out/new_input.npz')['train']
new_test_data = np.load('out/new_input.npz')['test']

In [38]:
model.fit(new_train_data[:, :-1], train_targets)

LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [39]:
preds = model.predict(new_test_data[:, :-1])
# np.savez('out/preds.npz',preds=preds)

In [40]:
print(accuracy_score(preds, test_targets))

0.740684882896


In [41]:
print(confusion_matrix(test_targets, preds))

[[7736 5097]
 [ 749 8962]]
